In [2]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt


SEED = 1234
tf.random.set_seed(SEED)

cwd = os.getcwd() #get current working directory
physical_devices = tf.config.list_physical_devices('GPU') 

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!pip install git+https://github.com/rstrudel/segmenter.git

In [ ]:
#!pip install --ignore-installed PyYAML 

From https://blog.tensorflow.org/2018/07/an-introduction-to-biomedical-image-analysis-tensorflow-dltk.html

Using a TFRecords database: For most deep learning problems on image volumes, the database of training examples is too large to fit into memory. The TFRecords format allows to serialise training examples and store them on disk with quick write access (i.e. parallel data reads):

In [ ]:
!mkdir -p '/content/drive/My Drive/Unzipped'
%cd '/content/drive/My Drive/Unzipped' 
!unzip '/content/drive/My Drive/Dataset/brats2019_slice_testing_tfrecords' 
!unzip '/content/drive/My Drive/Dataset/brats2019_slice_training_tfrecords' 
!unzip '/content/drive/My Drive/Dataset/brats2019_slice_validation_tfrecords' 

/content/drive/My Drive/Unzipped
Archive:  /content/drive/My Drive/Dataset/brats2019_slice_testing_tfrecords.zip
replace brats2019_slice_testing_tfrecords/brats2019_slice_testing.tfrecords? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Archive:  /content/drive/My Drive/Dataset/brats2019_slice_training_tfrecords.zip
replace brats2019_slice_training_tfrecords/brats2019_slice_training.tfrecords? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Archive:  /content/drive/My Drive/Dataset/brats2019_slice_validation_tfrecords.zip
replace brats2019_slice_validation_tfrecords/brats2019_slice_validation.tfrecords? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


# RUNNING


In [3]:
#set environment variables
os.environ['DATASET'] = "/content/drive/My Drive/Dataset"
os.environ['FORCE_CUDA'] = "1"

In [ ]:
#if you want to clone the repo
#!git clone https://github.com/rstrudel/segmenter.git

In [ ]:
#%cd /content/drive/My Drive/Segmenter/segmenter

Col codice che segue vorremmo prendere i file di tipo tfrecords e decodificarli per ottenerne le immagini.
Le colormaps si possono trovare al sito https://scipy-cookbook.readthedocs.io/items/Matplotlib_Show_colormaps.html


In [ ]:
#!pip install tensorflow tensorflow_addons pillow numpy matplotlib

In [20]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import numpy as np

#dataset_folder = os.path.join(cwd, "drive/My Drive/Unzipped/");
#dataset_name = os.path.join(dataset_folder, "brats2019_slice_training_tfrecords/brats2019_slice_training.tfrecords")
dataset_name = os.path.join('/content/drive/My Drive/Unzipped/', "brats2019_slice_training_tfrecords/brats2019_slice_training.tfrecords")
raw_dataset = tf.data.TFRecordDataset(dataset_name)
raw_dataset 
for raw_record in raw_dataset.take(20): #per capire la struttura del file tfrecord
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    #print(example)
    #print(repr(raw_record))


    parsed = tf.train.Example.FromString(raw_record.numpy())
    feature = parsed.features.feature

    raw_img_flair_mri = feature['flair_mri'].bytes_list.value[0]
    raw_img_seg = feature['seg'].bytes_list.value[0]
    raw_img_t1_mri = feature['t1_mri'].bytes_list.value[0]
    raw_img_t1ce_mri = feature['t1ce_mri'].bytes_list.value[0]
    raw_img_t2_mri = feature['t2_mri'].bytes_list.value[0]

    img_flair_mri = tf.image.decode_png(raw_img_flair_mri)
    img_t1_mri = tf.image.decode_png(raw_img_t1_mri)
    img_t1ce_mri = tf.image.decode_png(raw_img_t1ce_mri)
    img_t2_mri = tf.image.decode_png(raw_img_t2_mri)
    img_seg = tf.image.decode_png(raw_img_seg)

    f, splt = plt.subplots(1,5, figsize = (25.0, 25.0)) 

    
    splt[0].imshow(tf.squeeze(img_flair_mri), cmap='jet', vmin=0, vmax=255)
    splt[1].imshow(tf.squeeze(img_t1_mri), cmap='jet')
    splt[2].imshow(tf.squeeze(img_t2_mri), cmap='jet')
    splt[3].imshow(tf.squeeze(img_t1ce_mri), cmap='jet')
    splt[4].imshow(tf.squeeze(img_seg), cmap='jet')



Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#only once, if you need to download the ade20k dataset
#!python -m segm.scripts.prepare_ade20k $DATASET

In [ ]:
#!python -m segm.inference --model-path "/content/drive/My Drive/Models/Pre-trained/PASCAL - Seg-L-Mask16/checkpoint.pth" -i images/ -o segmaps/